In [19]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt

In [20]:
obs = xr.DataArray(
    np.random.normal(0, 0.8, size=(3, 3)),
    dims=["x", "y"],
    coords={"x": range(3), "y": range(3)}
)

In [24]:
resamples = xr.DataArray(
    np.random.normal(0.1, 1.5, size=(3, 3, 10)),
    dims=["x", "y", "iter"],
    coords={"x": range(3), "y": range(3), "iter": range(10)}
)

In [25]:
def get_quantile(value, sample):
    """
    Get the quantile of value relative to sample
    
    value: float
    sample: array
    """
    return np.searchsorted(np.sort(sample), value) / len(sample)

In [26]:
# Example output for x=y=0
get_quantile(
    obs.sel(x=0, y=0),
    resamples.sel(x=0, y=0)
)

0.3

In [48]:
xr.apply_ufunc(
    get_quantile,
    obs,
    resamples,
    input_core_dims=[["iter"]]
)

ValueError: input_core_dims must be None or a tuple with the length same to the number of arguments. Given 1 input_core_dims: [['iter']],  but number of args is 2.

In [54]:
def get_percentile(obs, bootstrap):
    """Returns the percentile of obs in bootstrap"""
    if np.isnan(obs):
        return np.nan
    else:
        return np.searchsorted(np.sort(bootstrap), obs) / len(bootstrap)

In [59]:
def apply_percentile(obs_da, bs_da, absolute=False, lat_name="y", lon_name="x"):
    """
        Gets the percentiles of obs_da in bs_da.
        This function is slow and I'm sure could be improved.
    """
    obs_da = obs_da.sortby(lat_name, ascending=True) # need lats in ascending order: https://stackoverflow.com/a/53175095
    bs_da = bs_da.sortby(lat_name, ascending=True)
    
    if absolute: # for e.g. magnitude of change point, we want the percentile of the absolute change
        obs_da = np.abs(obs_da)
        bs_da = np.abs(bs_da)
    
    obs_stack = obs_da.stack(point=(lat_name, lon_name)).groupby('point') # stack lat and lon onto new dimension 'point', then group by point
    bs_stack = bs_da.stack(point=(lat_name, lon_name)).groupby('point')

    percentiles = xr.apply_ufunc(get_percentile, obs_stack, bs_stack, input_core_dims=[[], ['iter']], output_core_dims=[[]], dask='allowed')
    ds = percentiles.unstack('point').sortby(lat_name, ascending=False) # unstack and sort lats to original ordering
    
    return ds

In [61]:
apply_percentile(obs, resamples).sel(x=0, y=0)

<xarray.DataArray ()>
array(0.3)
Coordinates:
    y        int64 0
    x        int64 0